In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import magic

In [2]:
# file_path = "../processed_data/pbmc10k_SEACells_summary.h5ad"
file_path = "../processed_data/GSE130973_SEACells_summary.h5ad"

adata = sc.read(file_path)

In [3]:
magic_op = magic.MAGIC()
approx_magic = magic_op.fit_transform(adata, genes='all_genes')

Calculating MAGIC...
  Running MAGIC on 15457 cells and 30867 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...


/home/genos/.local/lib/python3.8/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(


    Calculated PCA in 13.45 seconds.
    Calculating KNN search...
    Calculated KNN search in 9.07 seconds.
    Calculating affinities...
    Calculated affinities in 8.22 seconds.
  Calculated graph and diffusion operator in 30.75 seconds.
  Running MAGIC with `solver='exact'` on 30867-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
  Calculating imputation...
  Calculated imputation in 87.27 seconds.
Calculated MAGIC in 119.42 seconds.


In [4]:
sc.pp.highly_variable_genes(approx_magic, n_top_genes=2000, subset=True)

In [5]:
#approx_magic umap
sc.tl.pca(approx_magic, n_comps=100, use_highly_variable=True)

approx_magic.obsm['X_pca'] = approx_magic.obsm['X_pca'][:, 0:30] # after getting optimal number of PCs
approx_magic.varm['PCs'] = approx_magic.varm['PCs'][:, 0:30]
sc.pp.neighbors(approx_magic, n_neighbors=30, use_rep='X_pca', metric='euclidean', key_added='neighbors_30')

In [6]:
sc.tl.umap(approx_magic, neighbors_key='neighbors_30', min_dist=0.1)

In [7]:
# adata.layers['MAGIC'] = approx_magic.X
adata.obsm['X_magic_pca'] = approx_magic.obsm['X_pca']
adata.obsm['X_magic_umap'] = approx_magic.obsm['X_umap']

In [8]:
adata.layers['log_norm'] = adata.X.copy()

In [9]:
adata.var

,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length,feature_type,highly_variable,means,dispersions,dispersions_norm
ENSG00000243485,True,MIR1302-2HG,NCBITaxon:9606,gene,623,lncRNA,False,1.000000e-12,NaN,NaN
ENSG00000237613,True,FAM138A,NCBITaxon:9606,gene,888,lncRNA,False,1.000000e-12,NaN,NaN
ENSG00000186092,True,OR4F5,NCBITaxon:9606,gene,2618,protein_coding,False,1.000000e-12,NaN,NaN
ENSG00000238009,False,ENSG00000238009,NCBITaxon:9606,gene,629,lncRNA,False,1.753778e-03,-4.546900,-1.053845
ENSG00000239945,True,ENSG00000239945,NCBITaxon:9606,gene,1319,lncRNA,False,1.000000e-12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,False,MT-ND4L,NCBITaxon:9606,gene,297,protein_coding,False,5.563127e-01,-1.459180,-0.092909
ENSG00000198886,False,MT-ND4,NCBITaxon:9606,gene,1378,protein_coding,False,2.956016e+00,1.260410,0.000050
ENSG00000198786,False,MT-ND5,NCBITaxon:9606,gene,1812,protein_coding,False,1.236362e+00,0.019244,-0.362547
ENSG00000198695,False,MT-ND6,NCBITaxon:9606,gene,525,protein_coding,False,6.324706e-02,-3.498859,0.004616


In [36]:
# Save the modified AnnData object
# adata.write('../processed_data/pbmc10k_SEACells_MAGIC.h5ad', compression='gzip')
adata.write('../processed_data/GSE130973_SEACells_MAGIC.h5ad', compression='gzip')

In [ ]:
approx_magic

AnnData object with n_obs × n_vars = 15457 × 2000
    obs: 'age', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'donor_id', 'suspension_type', 'Cluster', 'Celltype', 'tissue_type', 'sample_id', 'library_id', 'library_preparation_batch', 'library_sequencing_run', 'alignment_software', 'manner_of_death', 'sample_source', 'sample_collection_method', 'institute', 'sampled_site_condition', 'sample_preservation_method', 'sequenced_fragment', 'reference_genome', 'cell_enrichment', 'gene_annotation_version', 'cell_type', 'assay', 'disease', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'SEACell'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'highly_variable', 'means', 'dispersions', 'dispersi

In [17]:
approx_magic.var

,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length,feature_type,highly_variable,means,dispersions,dispersions_norm
ENSG00000188290,False,HES4,NCBITaxon:9606,gene,961,protein_coding,True,0.385177,0.761288,2.028188
ENSG00000186891,False,TNFRSF18,NCBITaxon:9606,gene,789,protein_coding,True,0.268602,-0.442180,3.098512
ENSG00000186827,False,TNFRSF4,NCBITaxon:9606,gene,1039,protein_coding,True,0.120312,-0.838435,2.697751
ENSG00000179403,False,VWA1,NCBITaxon:9606,gene,1483,protein_coding,True,0.048009,-1.430383,2.099072
ENSG00000197530,False,MIB2,NCBITaxon:9606,gene,1455,protein_coding,True,0.157099,-1.569897,1.957971
...,...,...,...,...,...,...,...,...,...,...
ENSG00000160180,False,TFF3,NCBITaxon:9606,gene,452,protein_coding,True,0.619089,3.555539,3.606666
ENSG00000160191,False,PDE9A,NCBITaxon:9606,gene,1724,protein_coding,True,0.015172,-1.810278,1.714856
ENSG00000183255,False,PTTG1IP,NCBITaxon:9606,gene,755,protein_coding,True,0.238921,-1.989137,1.533963
ENSG00000160255,False,ITGB2,NCBITaxon:9606,gene,692,protein_coding,True,0.164043,-0.086117,3.458624


In [10]:
# adata.X as pd.DataFrame
X = pd.DataFrame(adata.X.toarray(), index=adata.obs.index, columns=adata.var.index)

In [12]:
# iterate for genes 'ENSG00000160191' and 'ENSG00000186827' until t = 5
magic_op

MAGIC(knn_max=15)